In [1]:
import requests
import pandas as pd
import numpy as np
import xarray as xr
import sqlite3
import os
from datetime import datetime
import cdsapi
import csv

In [2]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table : ", error)

# Connexion à la bdd piezo
Construction d'un dataframe 'df_piezo' à partir de la requête sur la bdd

In [20]:
region = "OCC"

In [21]:
connexion = sqlite3.connect('./data/liste_piezos.db')
curseur = connexion.cursor()
requete = f"""
    SELECT code_bss, date_debut, date_fin, nb_mesures_piezo, latitude, longitude 
        FROM data_piezo
    INNER JOIN code_region
    ON code_dpt = departement
    WHERE code_region IS '{region}'
    AND date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""

df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])

In [22]:
df_piezo

,date_debut,date_fin,nb_mesures_piezo,latitude,longitude
code_bss,,,,,
09654X0569_CNRP20,2004-03-14,2022-05-24,6243,43.848274,4.607080
08592X0175_FOR170,2008-11-17,2020-12-31,618,44.557278,2.251595
09892X0679_EXH70,2004-02-01,2022-05-24,6544,43.695576,3.327317
10615X0047_F0,2009-02-27,2022-04-07,4408,43.054528,2.921968
09648X0083_F90-1,1995-01-31,2022-05-02,7659,43.744145,4.230897
...,...,...,...,...,...
08576X0022_HY,2009-04-08,2019-06-04,3398,44.506078,1.654776
08577X0029_HY,2009-04-08,2021-01-04,4291,44.521516,1.668573
09635X0003_FHY,1997-06-01,2022-05-24,8323,43.759836,3.624061


# Fonction pour lancer les requêtes

In [23]:
def get_on_cds_api(dataframe_piezo, piezo, path):
    # Initialisation de la connexion à l'API CDS
    # wait_until_complete = False : le programme Python n'attend pas que l'API réponde (issu de la libraie asyncio)
    # delete = False : le fichier créé sur le serveur source n'est pas supprimé automatiquement 
    
    date_debut = dataframe_piezo.loc[piezo,'date_debut'].year
    date_fin = dataframe_piezo.loc[piezo,'date_fin'].year
    
    # La limite du nombre de données est de 120000. Avec 10 variables, 31 jours, et 12 mois, on n epeut récupérer que 32 années.
    # Si l'interval d'année est supérieur à 32, on réduit la plage de temps à date de fin - 31 ans pour avoir 32 années max.
    if date_fin - date_debut >= 32:
        date_debut = date_fin - 31
    
    c = cdsapi.Client(wait_until_complete=False, delete=False)
    
    # Données à récupérer auprès de l'API
    r = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '2m_temperature', 'convective_precipitation', 'convective_rain_rate',
                    'convective_snowfall', 'large_scale_rain_rate', 'large_scale_snowfall', 'snowfall',
                    'large_scale_snowfall_rate_water_equivalent', 'potential_evaporation',
                    'total_precipitation',
                ],
                'year': [str(date) for date in (np.arange(date_debut, date_fin))],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': '00:00',
                'area': [
                    df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"], df_piezo.loc[piezo,"latitude"], df_piezo.loc[piezo,"longitude"],
                ],
            })    
    return r

## Lancer les requêtes auprès de l'API
!!! NE LANCER QU'UNE SEULE FOIS POUR NE PAS CREER DES DOUBLONS DE REQUETES !!!

Un dictionnaire est créé avec en clé le code_bss du piezo et en valeur l'id de la requête

Ne pas lancer la cellule ci-dessous si des requetes sont en cours. 

In [24]:
# Parcourir les codes_bss des piezos dans la liste précédemment créé :
dict_bss = {}

In [25]:
go_requete = input("""
            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
""")
                   
if go_requete == 'req' :
                      
    for piezo in df_piezo.index:
        path_piezo = 'data/era5/' + piezo
        path_nc = f"./{path_piezo}.nc"
        path_pkl = f"{path_piezo}.pkl"

        if os.path.isfile(path_nc) : #si le fichier nc existe on le converti en pkl
            print(f"Le fichier {path_nc} existe.")
            df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
            print(f"Fichier {piezo}.nc converti en fichier pickle")
            os.remove(path_nc)
            print(f"Fichier {path_nc} supprimé")

        elif os.path.isfile(path_pkl) : #si le fichier pkl existe, on affiche qu'il existe
            print(f"Le fichier {piezo}.pkl existe déjà.")
            # puis on le supprime de la liste
            df_piezo.drop(index=piezo, axis=1)

        else : # Sinon on lance une requête auprès de l'API
            result = get_on_cds_api(df_piezo, piezo, path_nc)
            # récupération de l'id de la requête
            dict_bss[piezo] = result.reply["request_id"]
            print(f"{datetime.now()} : chargement du fichier {piezo}.nc en cours ({dict_bss[piezo]}).")
            
            # Enregistrement de l'id de requete dans un fichier csv
            if os.path.isfile('./data/hist_requetes.csv'): # Si le fichier existe                
                with open('./data/hist_requetes.csv', 'a', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
            else : # Si le fichier n'existe pas
                with open('./data/hist_requetes.csv', 'w', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    writer.writerow(["code_bss", "id_requete"])
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
else : 
    print("Aucune requête n'a été lancée.")


            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
req


2022-07-21 13:33:45,391 INFO Welcome to the CDS
2022-07-21 13:33:45,391 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,468 INFO Welcome to the CDS
2022-07-21 13:33:45,469 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,530 INFO Welcome to the CDS
2022-07-21 13:33:45,531 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,591 INFO Welcome to the CDS
2022-07-21 13:33:45,593 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:33:45.431814 : chargement du fichier 09654X0569_CNRP20.nc en cours (14e7b9c4-eae1-401c-ad9c-46363f9a7347).
2022-07-21 13:33:45.502896 : chargement du fichier 08592X0175_FOR170.nc en cours (8c0fdca8-d253-4caa-b7e5-9968f12913c0).
2022-07-21 13:33:45.563774 : chargement du fichier 09892X0679_EXH70.nc en cours (d9b727dc-2adb-441b-9b39-7b2768d8616c).


2022-07-21 13:33:45,677 INFO Welcome to the CDS
2022-07-21 13:33:45,679 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:33:45,740 INFO Welcome to the CDS
2022-07-21 13:33:45,742 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:33:45.651273 : chargement du fichier 10615X0047_F0.nc en cours (236db60c-c39b-4415-96bf-a105f6a832e3).
2022-07-21 13:33:45.712718 : chargement du fichier 09648X0083_F90-1.nc en cours (30f5a1d4-c96c-47e9-8eea-a3bf82fa6ef6).
2022-07-21 13:33:45.779320 : chargement du fichier 10906X0039_C2-1.nc en cours (6020aefe-70e0-496c-85ff-77cc5be4a0c9).


...

2022-07-21 13:34:06.136964 : chargement du fichier 10882X0215_ADOUX.nc en cours (0fbe163b-302a-4522-970b-ba4bdf9f839f).
2022-07-21 13:34:06.254614 : chargement du fichier 09625X0231_LADOUX.nc en cours (24fe87a3-85d3-49f4-837e-8ce4fbd41b88).
2022-07-21 13:34:06.316984 : chargement du fichier 09126X0275_RABY.nc en cours (4a2ab5bd-d692-4ed0-b773-9d7f34422098).


2022-07-21 13:34:06,345 INFO Welcome to the CDS
2022-07-21 13:34:06,346 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:34:06,411 INFO Welcome to the CDS
2022-07-21 13:34:06,411 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:34:06,472 INFO Welcome to the CDS
2022-07-21 13:34:06,473 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-21 13:34:06,534 INFO Welcome to the CDS
2022-07-21 13:34:06,534 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:34:06.383898 : chargement du fichier 08576X0022_HY.nc en cours (f8d9c845-f14f-4581-b019-8317ef8efcc4).
2022-07-21 13:34:06.444614 : chargement du fichier 08577X0029_HY.nc en cours (bf1e711b-7018-4e40-b203-0568fe291213).
2022-07-21 13:34:06.505687 : chargement du fichier 09635X0003_FHY.nc en cours (e4a81d22-7edf-4ce5-839a-5fef319a6cca).
2022-07-21 13:34:06.566919 : chargement du fichier 10165X0021_CAUVY.nc en cours (980333d9-614c-43f6-a9ac-6758112e1536).


2022-07-21 13:34:06,595 INFO Welcome to the CDS
2022-07-21 13:34:06,595 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-21 13:34:06.627972 : chargement du fichier 10581X0201_SAGRAZ.nc en cours (6571df72-4bc2-4af1-9742-d8cda20df656).


## Récupérer automatiquement les requêtes abouties 
Peut être lancé à tout moment pour récupérer des fichiers téléchargés 

In [3]:
# Chargement du dictionnaire à partir du csv d'historiques de requetes
with open('./data/hist_requetes.csv', mode='r') as f:
    reader = csv.reader(f, delimiter=";")
    # Permet de sortir la ligne d'en-tête du dictionnaire
    headings = next(reader)
    dict_bss_from_csv = {rows[0]:rows[1] for rows in reader}

nbre_fichier_OK = 0

    
for piezo in dict_bss_from_csv.copy().keys():
    path_piezo = 'data/era5/' + piezo
    path_nc = f"./{path_piezo}.nc"
    path_pkl = f"{path_piezo}.pkl"

    if not os.path.isfile(path_nc) :
        try:
            reply = dict(request_id=dict_bss_from_csv[piezo]) # request_id from above
            new_client = cdsapi.Client(wait_until_complete=False, delete=False)
            result = cdsapi.api.Result(new_client, reply)
            result.update()
            reply = result.reply
            if reply['state'] == 'completed':
                nbre_fichier_OK += 1
                result.download(path_nc)
                print(f"{datetime.now()} : Fichier {piezo}.nc téléchargé.")
                df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
                print(f"Fichier {piezo}.nc converti en fichier pickle")
                os.remove(path_nc)
                print(f"Fichier {path_nc}.nc supprimé")
                # Si le fichier a été téléchargé on le supprime de la liste du dictionnaire
                del dict_bss[piezo]
        except:
            #print(f"Récupération de l'ID {dict_bss_from_csv[piezo]} impossible.")
            pass
if nbre_fichier_OK == 0:
    print("Aucun fichier de récupéré.")
else :
    print(f"{nbre_fichier_OK} fichiers ont été téléchargés.")

# Mise à jour de la base de donnéees (champ fichier_ERA5)
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)
print("Base de données à jour")

2022-07-25 11:41:32,961 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data6/adaptor.mars.internal-1658480648.9372804-23485-7-931f7632-5bf5-42ad-bfe4-bf585a81c1f6.nc to ./data/era5/10553X0026_F.nc (226K)
2022-07-25 11:41:33,731 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data3/adaptor.mars.internal-1658481400.132289-1973-18-52e49fed-50d6-463e-96db-822012823eb7.nc to ./data/era5/10092X0166_F.nc (217.4K)
2022-07-25 11:41:34,063 INFO Downloading https://download-0016.copernicus-climate.eu/cache-compute-0016/cache/data4/adaptor.mars.internal-1658483723.6151054-32144-5-a338c757-0538-421c-90f3-af058578b4e1.nc to ./data/era5/09306X0286_F.nc (200.3K)
2022-07-25 11:41:34,433 INFO Downloading https://download-0013-clone.copernicus-climate.eu/cache-compute-0013/cache/data4/adaptor.mars.internal-1658481989.0020401-2073-12-49278831-0bfe-4fdd-82dc-ce1a054dac3e.nc to ./data/era5/08565X0007_F.nc (106.2K)
2022-

2022-07-25 11:41:34.820810 : Fichier 08565X0007_F.nc téléchargé.
Fichier 08565X0007_F.nc converti en fichier pickle
Fichier ./data/era5/08565X0007_F.nc.nc supprimé


2022-07-25 11:41:35,024 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data3/adaptor.mars.internal-1658482697.5337512-22151-19-a654d1ef-5bfe-4040-b167-a85fe7d827ba.nc to ./data/era5/10603X0027_F.nc (157.5K)
2022-07-25 11:41:35,441 INFO Download rate 377.9K/s
2022-07-25 11:41:35,543 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data2/adaptor.mars.internal-1658483766.8941596-31819-16-1b6e349f-a2b5-41d9-b58e-4a9ad57ad256.nc to ./data/era5/09838D0727_F.nc (217.4K)


2022-07-25 11:41:35.442446 : Fichier 10603X0027_F.nc téléchargé.
Fichier 10603X0027_F.nc converti en fichier pickle
Fichier ./data/era5/10603X0027_F.nc.nc supprimé


2022-07-25 11:41:35,870 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data2/adaptor.mars.internal-1658483723.540129-25749-4-c569fdaa-c4d1-4b45-b20b-e2fd7695da51.nc to ./data/era5/09577X0047_F.nc (226K)
2022-07-25 11:41:36,205 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data9/adaptor.mars.internal-1658483303.6080184-26431-17-f3cc5391-435b-4bb2-af33-9d8e60d926f6.nc to ./data/era5/09395X0067_F.nc (157.5K)
2022-07-25 11:41:36,537 INFO Downloading https://download-0004-clone.copernicus-climate.eu/cache-compute-0004/cache/data9/adaptor.mars.internal-1658480634.347901-6137-17-06ff1fad-f961-466a-8146-ab8e2f62d9cc.nc to ./data/era5/09536X0008_F.nc (157.5K)
2022-07-25 11:41:36,822 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data3/adaptor.mars.internal-1658480639.4628153-11920-19-d7ded761-d4f9-497c-9a20-5b6fa27dd43a.nc to ./data/era5/09301X0028_F.nc (226K)


...

2022-07-25 11:42:03.215368 : Fichier 09388X0052_VIGNOT.nc téléchargé.
Fichier 09388X0052_VIGNOT.nc converti en fichier pickle
Fichier ./data/era5/09388X0052_VIGNOT.nc.nc supprimé


2022-07-25 11:42:03,587 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data3/adaptor.mars.internal-1658526739.7026966-11882-5-145f0222-9d0e-42ee-9382-14d02bb23215.nc to ./data/era5/09876X0212_PORT.nc (149K)
2022-07-25 11:42:03,890 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/adaptor.mars.internal-1658526739.9242733-5826-17-83be31c3-1393-40cb-a07d-fb99def1ed8b.nc to ./data/era5/10976X0058_RIMBAU.nc (157.5K)
2022-07-25 11:42:04,204 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data6/adaptor.mars.internal-1658526341.13916-17470-11-348000ea-9bf6-4c27-ab1c-bb227aaa0aba.nc to ./data/era5/09882X0208_FONCAU.nc (149K)
2022-07-25 11:42:04,558 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars.internal-1658527615.0461516-7109-4-8c140dd6-f2a8-4e50-9a8c-3c3be8481160.nc to ./data/era5/09908X0174_MASJAU.nc

2022-07-25 11:42:06.762771 : Fichier BSS003PAFM_X.nc téléchargé.
Fichier BSS003PAFM_X.nc converti en fichier pickle
Fichier ./data/era5/BSS003PAFM_X.nc.nc supprimé


2022-07-25 11:42:07,179 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data7/adaptor.mars.internal-1658521939.2463582-14574-15-0fbe163b-302a-4522-970b-ba4bdf9f839f.nc to ./data/era5/10882X0215_ADOUX.nc (106.2K)
2022-07-25 11:42:07,511 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data0/adaptor.mars.internal-1658521213.204858-31409-7-24fe87a3-85d3-49f4-837e-8ce4fbd41b88.nc to ./data/era5/09625X0231_LADOUX.nc (149K)
2022-07-25 11:42:07,843 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data4/adaptor.mars.internal-1658528314.053892-18054-10-4a2ab5bd-d692-4ed0-b773-9d7f34422098.nc to ./data/era5/09126X0275_RABY.nc (106.2K)
2022-07-25 11:42:08,127 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data2/adaptor.mars.internal-1658528077.291807-18398-11-f8d9c845-f14f-4581-b019-8317ef8efcc4.nc to ./data/era5/08576X0022_HY.nc (89K)


106 fichiers ont été téléchargés.
Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


## Jointure era5 et piezo

In [4]:
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"./data/era5") if _.endswith(r".pkl")], name="era5")
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")
piezos_communs = np.intersect1d(bss_era5, bss_piezo)

path = "./data/combined_data/"
# Le fichier doit avoir un minimum de 100 mesures
nb_min_val_com = 100

for piezo in piezos_communs :
    if not os.path.isfile(f'{path}/comb_{piezo}.csv') : #si le fichier n'existe pas 
        df_piezo = pd.read_csv(f"./data/piezo/{piezo}.csv", sep = ";", index_col=0, parse_dates=True)
        # Group_by date_mesure dans le cas de plusieurs mesures le même jour ce qui provoquerait une erreur
        df_piezo = pd.DataFrame(df_piezo.groupby("date_mesure").mean())
        df_piezo = df_piezo.resample('1D').asfreq()
        df_era5 = pd.read_pickle(f"./data/era5/{piezo}.pkl").set_index('time').drop(["latitude", "longitude"], axis=1)
        if "expver" in df_era5.columns :
            df_era5 = df_era5.loc[df_era5["expver"]==1,:].drop("expver", axis=1)
        df = pd.merge(df_piezo, df_era5, how='inner', left_index=True, right_index=True)
        nb_com_obs = df.shape[0]
        if nb_com_obs >= nb_min_val_com :
            df.to_csv(f"data/combined_data/comb_{piezo}.csv" ,sep=";", index=True)
            print(f"Le fichier comb_{piezo}.csv a été créé et contient {nb_com_obs} obseravtions.")
        else :
            print(f"Le piezo {piezo} ne contient pas assez de données (seulement {nb_com_obs}).")
    else : 
        #print(f"Le fichier comb_{piezo}.csv existe déjà.")
        pass
        
# Mise à jour bdd
requete_fichier_combined = f"""
        UPDATE data_piezo
        SET fichier_combined =
        (CASE WHEN code_bss IN {tuple(piezos_communs)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_combined)
print("Base de données à jour")

Le fichier comb_08565X0007_F.csv a été créé et contient 4109 obseravtions.
Le fichier comb_09301X0028_F.csv a été créé et contient 9388 obseravtions.
Le fichier comb_09388X0052_VIGNOT.csv a été créé et contient 11323 obseravtions.
Le fichier comb_09388X0114_MAISON.csv a été créé et contient 7854 obseravtions.
Le fichier comb_09394X0003_FONTAI.csv a été créé et contient 1552 obseravtions.
Le fichier comb_09567X0219_F.csv a été créé et contient 9144 obseravtions.
Le fichier comb_09635X0009_FONTAN.csv a été créé et contient 9174 obseravtions.
Le fichier comb_09656X0175_RODIL.csv a été créé et contient 9481 obseravtions.
Le fichier comb_09888X0111_LACAN.csv a été créé et contient 5949 obseravtions.
Le fichier comb_09921X0055_CAMBON.csv a été créé et contient 6175 obseravtions.
Le fichier comb_10364X0001_S.csv a été créé et contient 5826 obseravtions.
Le fichier comb_10384X0048_F.csv a été créé et contient 4927 obseravtions.
Le fichier comb_10388X0017_STALIN.csv a été créé et contient 11323

# Récupération manuelle

In [ ]:
reply = dict(request_id="54c52ee3-eee4-481d-b72e-fecd315085cb")
new_client = cdsapi.Client(wait_until_complete=False, delete=False)
result = cdsapi.api.Result(new_client, reply)
result.update()
reply = result.reply
if reply['state'] == 'completed':
    result.download('file.nc')

In [ ]:
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)

In [ ]:
update table set
    columnx = (case when condition then 25 else columnx end)